In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
Map = geemap.Map()
roi = geemap.shp_to_ee(r"D:\地理数据\四套中国地图：有审图号\中国标准行政区划数据GS（2024）0650号\shp格式\中国_省.shp") \
                .filter(ee.Filter.eq('name', '江西省')) \
                .geometry() \
                .buffer(1000)

In [ ]:
for year in [2020, 2024]:
    for month in range(1, 13):
        startdate = ee.Date(f'{year}-{month:02d}-01')
        enddate = startdate.advance(1, 'month')
        lai = ee.ImageCollection('MODIS/061/MCD15A3H') \
                    .filterDate(startdate, enddate) \
                    .select('Lai') \
                    .map(lambda img: img.multiply(0.1)) \
                    .max() \
                    .clip(roi)
        
        if year == 2024:
            gpp = ee.ImageCollection("MODIS/061/MOD17A2H") \
                        .filterDate(startdate, enddate) \
                        .select('Gpp') \
                        .map(lambda img: img.multiply(0.0001)) \
                        .sum() \
                        .clip(roi)
            geemap.download_ee_image(gpp, rf'D:\任务归档\NPP\otherdata\gpp\江西省{year}年GPP数据\GPP{year}_{month:02d}.tif', region=roi, scale=500, crs='EPSG:4527')

        
        geemap.download_ee_image(lai, rf'D:\任务归档\NPP\otherdata\lai\江西省{year}年LAI数据\LAI{year}_{month:02d}.tif', region=roi, scale=500, crs='EPSG:4527')
        


In [ ]:
gpp